In [21]:
import csv
import random

def generate_unique_colors(num_colors):
    """
    Generates a list of unique, visually distinct RGB colors.

    Args:
        num_colors: The number of unique colors to generate.

    Returns:
        A list of strings, where each string is an RGB color code (e.g., "#FF0000").
    """
    colors = set() # Use a set to guarantee uniqueness.

    # Simple method to generate distinct colors.  Not *perfectly* distributed,
    # but good enough for this purpose.  Better methods exist for perceptual
    # uniformity, but are more complex.
    while len(colors) < num_colors:
        # Generate random RGB values
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)

        # Convert to hex string and add to set
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)  # Use hex format
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n):
    """
    Generates an HTML table, coloring 8x8 blocks based on content.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    # Calculate the number of colors needed
    num_colors = (dim // 8) * (dim // 8)  # Number of 8x8 blocks
    if num_colors == 0:
       num_colors = 1 # if there is only one block 8x8.

    # Generate unique colors
    colors = generate_unique_colors(num_colors)

    block_signatures = {}
    next_color_index = 0

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse; }
        td { border: 1px solid black; padding: 5px; text-align: center;color:white }
    </style>
</head>
<body>
    <table>
"""

    for i in range(dim):
        html_string += "        <tr>\n"
        for j in range(dim):
            i_block = (i // 8) * 8
            j_block = (j // 8) * 8

            block_signature = []
            for row_index in range(i_block, min(i_block + 8, dim)):
                for col_index in range(j_block, min(j_block + 8, dim)):
                    block_signature.append(table_data[row_index][col_index].strip().lstrip('+-'))
            block_signature = tuple(sorted(block_signature))

            if block_signature in block_signatures:
                color = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)] # use len(colors) and not num_colors
                block_signatures[block_signature] = color
                next_color_index += 1

            cell_value = table_data[i][j]
            html_string += f"            <td style='background-color: {color};'>{cell_value}</td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")
    
def main():
    csv_filename = input("Enter the filename of the Cayley-Dickson CSV table: ")
    html_filename = input("Enter the desired filename for the output HTML table (e.g., table.html): ")
    


def main():
    

    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}.html"
    generate_colored_html_table(csv_filename, html_filename, n)

if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n7.html'


In [18]:
import csv
import random

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n):
    """
    Generates an HTML table, coloring 8x8 blocks based on content similarity
    (INCLUDING signs), and without the first row/column.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // 8) * (dim // 8)  # Number of 8x8 blocks
    if num_colors == 0:
        num_colors = 1

    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse; }
        td { border: 1px solid black; padding: 5px; text-align: center; }
    </style>
</head>
<body>
    <table>
"""

    for i in range(0, dim, 8):
        for row_index in range(i, min(i + 8, dim)):
            html_string += "        <tr>\n"
            for j in range(0, dim, 8):
                # Create a "signature" for the 8x8 block (INCLUDING signs)
                block_signature = []
                for r in range(min(8, dim - row_index)):
                    for c in range(min(8, dim - (j))):
                        block_signature.append(table_data[row_index + r][j + c].strip()) #KEEP signs.
                block_signature = tuple(block_signature) # No need to sort.

                if block_signature in block_signatures:
                    color = block_signatures[block_signature]
                else:
                    color = colors[next_color_index % len(colors)]
                    block_signatures[block_signature] = color
                    next_color_index += 1

                for col_index in range(j, min(j + 8, dim)):
                    cell_value = table_data[row_index][col_index]
                    html_string += f"            <td style='background-color: {color};'>{cell_value}</td>\n"
            html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same.html"
    generate_colored_html_table(csv_filename, html_filename, n)

if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n4_same.html'


In [26]:
import csv
import random

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n,block_dimension):
    """
    Generates an HTML table, coloring 8x8 blocks based on *content*
    (ignoring signs), and without the first row/column.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // block_dimension) * (dim // block_dimension)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
               table {
  width: 100%; /* Ensure the table takes the full width of its container */
  border-collapse: collapse; /* Optional: Collapse borders for a cleaner look */
}

td {
  width: 60px; /* Set a fixed width for each cell */
  height: 40px; /* Set a fixed height for each cell */
  box-sizing: border-box; /* Include padding and border in the element's total width and height */
  text-align: center; /* Optional: Center the text within the cell */
  border: 1px solid #000; /* Optional: Add a border to the cells */
  color:white;
}
</style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // block_dimension) * block_dimension
            j_block = (j // block_dimension) * block_dimension

            # Create a "signature" for the 8x8 block (ignoring signs)
            block_signature = []
            for row_index in range(i_block, min(i_block + block_dimension, dim)):
                for col_index in range(j_block, min(j_block + block_dimension, dim)):
                    block_signature.append(table_data[row_index][col_index].strip().lstrip('+-')) # Remove signs
            block_signature = tuple(sorted(block_signature)) # Sort to make order-independent

            # Get color for this signature
            if block_signature in block_signatures:
                color = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                block_signatures[block_signature] = color
                next_color_index += 1

            # Generate the HTML for the cell
            cell_value = table_data[i][j]
            html_string += f"            <td style='background-color: {color};'>{cell_value}</td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():
    
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
        size=int(input("Enter the value of block size "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same_elements_block_{size}.html"
    block_dimension=2**size
    generate_colored_html_table(csv_filename, html_filename, n,block_dimension)


if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n10_same_elements_block_5.html'


In [1]:
import csv
import random
import re

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n):
    """
    Generates an HTML table, coloring 8x8 blocks based on *exact* content
    (including signs and order), and without the first row/column.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // 8) * (dim // 8)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse; }
        td { border: 1px solid black; padding: 5px; text-align: center; }
    </style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // 8) * 8
            j_block = (j // 8) * 8

            # Create a "signature" for the 8x8 block (INCLUDING signs and order)
            block_signature = []
            for row_index in range(i_block, min(i_block + 8, dim)):
                for col_index in range(j_block, min(j_block + 8, dim)):
                    block_signature.append(re.sub(r'\d', '',table_data[row_index][col_index])) # Keep signs and order removed .strip()
            block_signature = tuple(block_signature) # No sorting

            # Get color for this signature
            if block_signature in block_signatures:
                color = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                block_signatures[block_signature] = color
                next_color_index += 1

            # Generate the HTML for the cell
            cell_value = table_data[i][j]
            html_string += f"            <td style='background-color: {color};'>{cell_value}</td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same_signs.html"
    generate_colored_html_table(csv_filename, html_filename, n)

if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n8_same_signs.html'


In [3]:
import csv
import random

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n,block_dimension):
    """
    Generates an HTML table, coloring 8x8 blocks based on *exact* content
    (including signs and order), and without the first row/column.
    Replaces block content with a unique number if the block repeats.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // block_dimension) * (dim // block_dimension)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0
    block_number = 1  # Start numbering blocks from 1

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse;color:white }
        td { border: 1px solid black; padding: 5px; text-align: center; }
    </style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // block_dimension) * block_dimension
            j_block = (j // block_dimension) * block_dimension

            # Create a "signature" for the 8x8 block (INCLUDING signs and order)
            block_signature = []
            for row_index in range(i_block, min(i_block + block_dimension, dim)):
                for col_index in range(j_block, min(j_block + block_dimension, dim)):
                    block_signature.append(re.sub(r'\d', '',table_data[row_index][col_index]))#table_data[row_index][col_index].strip())
            block_signature = tuple(block_signature)

            # Get color and number for this signature
            if block_signature in block_signatures:
                color, number = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                number = block_number
                block_signatures[block_signature] = (color, number) # Store color AND number
                next_color_index += 1
                block_number += 1

            # Generate the HTML for the cell
            if (i, j) == (i_block, j_block):  # Top-left cell of the block
                html_string += f"            <td style='background-color: {color};'>\"{number}\"</td>\n"
            #else:
                #  Empty cell, but with the correct background color
                #html_string += f"            <td style='background-color: {color};'></td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")
def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
        size=int(input("Enter the block size : "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_sameN_dimension_{size}.html"
    block_dimension=2**size
    generate_colored_html_table(csv_filename, html_filename, n,block_dimension)

if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n7_sameN_dimension_0.html'


In [51]:
import csv
import random

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n):
    """
    Generates an HTML table, coloring 8x8 blocks based on *exact* content
    (including signs and order), and without the first row/column.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // 8) * (dim // 8)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0
    block_number = 1 
    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse; }
        td { border: 1px solid black; padding: 5px; text-align: center; }
    </style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // 8) * 8
            j_block = (j // 8) * 8

            # Create a "signature" for the 8x8 block (INCLUDING signs and order)
            block_signature = []
            for row_index in range(i_block, min(i_block + 8, dim)):
                for col_index in range(j_block, min(j_block + 8, dim)):
                    block_signature.append(table_data[row_index][col_index].strip().lstrip('+-')) # Keep signs and order
            block_signature = tuple(block_signature) # No sorting

            # Get color for this signature
            if block_signature in block_signatures:
                color,number = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                number = block_number
                block_signatures[block_signature] = (color, number)
                next_color_index += 1

            # Generate the HTML for the cell
            #cell_value = table_data[i][j]
            if (i, j) == (i_block, j_block):  # Top-left cell of the block
                html_string += f"            <td style='background-color: {color};'>\"{number}\"</td>\n"
            #else:
                #  Empty cell, but with the correct background color
                #html_string += f"            <td style='background-color: {color};'></td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same3N2.html"
    generate_colored_html_table(csv_filename, html_filename, n)

if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n11_same3N2.html'


In [49]:
import csv
import random

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n):
    """
    Generates an HTML table, coloring 8x8 blocks based on *exact* content
    (including signs and order), and without the first row/column.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // 8) * (dim // 8)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0
    block_number = 1 
    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse; }
        td { border: 1px solid black; padding: 5px; text-align: center; }
    </style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // 8) * 8
            j_block = (j // 8) * 8

            # Create a "signature" for the 8x8 block (INCLUDING signs and order)
            block_signature = []
            for row_index in range(i_block, min(i_block + 8, dim)):
                for col_index in range(j_block, min(j_block + 8, dim)):
                    block_signature.append(table_data[row_index][col_index].strip()[0]) # Keep signs and order
            block_signature = tuple(block_signature) # No sorting

            # Get color for this signature
            if block_signature in block_signatures:
                color,number = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                number = block_number
                block_signatures[block_signature] = (color, number)
                next_color_index += 1

            # Generate the HTML for the cell
            #cell_value = table_data[i][j]
            if (i, j) == (i_block, j_block):  # Top-left cell of the block
                html_string += f"            <td style='background-color: {color};'>\"{number}\"</td>\n"
            #else:
                #  Empty cell, but with the correct background color
                #html_string += f"            <td style='background-color: {color};'></td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same3NS.html"
    generate_colored_html_table(csv_filename, html_filename, n)

if __name__ == "__main__":
    main()

HTML table generated and saved to 'cayley_dickson_table_n11_same3NS.html'
